<a href="https://colab.research.google.com/github/hibabtl/DSAI-BATCH1/blob/main/Topical_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [29]:
import tensorflow as tf
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import json

# Load pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add the eos_token as the padding token
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

# Load and preprocess your custom dataset
dataset_path = '/content/drive/MyDrive/NLP PROJECT/Topical-Chat/conversations/test_freq.json'
with open(dataset_path, "r") as json_file:
    dataset = json.load(json_file)

In [30]:
# Create pairs of questions and answers
conversations = []
for conversation_id, conversation in dataset.items():
    conversation_history = []
    for turn in conversation["content"]:
        message = turn["message"][0] if isinstance(turn["message"], list) else turn["message"]
        conversation_history.append(message)

    # Create pairs of question (user) and answer (bot)
    for i in range(len(conversation_history) - 1):
        question = conversation_history[i]
        answer = conversation_history[i + 1]
        conversations.append({"question": question, "answer": answer})

# Tokenize and format data for training
input_data = [f"User: {pair['question']} Bot: {pair['answer']}" for pair in conversations]
tokenized_data = tokenizer(input_data, return_tensors="tf", padding=True, truncation=True)

# Prepare input and target tensors for training
input_ids = tokenized_data["input_ids"][:, :-1]  # Input tensor (remove last token)
target_ids = tokenized_data["input_ids"][:, 1:]   # Target tensor (shifted by one position)

# Create TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": input_ids, "attention_mask": tokenized_data["attention_mask"][:, :-1]}, target_ids))


In [32]:
# Define model and training parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

@tf.function
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        # Ensure we are using TensorFlow operations
        logits = model(inputs["input_ids"], attention_mask=inputs["attention_mask"]).logits
        loss = loss_fn(targets, logits)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training loop
epochs = 3
for epoch in range(epochs):
    for batch in dataset.shuffle(1000).batch(16):
        inputs, targets = batch
        loss = train_step(inputs, targets)
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {loss:.4f}")

# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/NLP PROJECT")

AttributeError: ignored

In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import DataLoader, Dataset
import json

In [2]:
# Load pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


In [3]:
# Add the eos_token as the padding token
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

# Load and preprocess your custom dataset
dataset_path = '/content/drive/MyDrive/NLP PROJECT/Topical-Chat/conversations/test_freq.json'
with open(dataset_path, "r") as json_file:
    dataset = json.load(json_file)

In [4]:
# Create pairs of questions and answers
conversations = []
for conversation_id, conversation in dataset.items():
    conversation_history = []
    for turn in conversation["content"]:
        message = turn["message"][0] if isinstance(turn["message"], list) else turn["message"]
        conversation_history.append(message)

    # Create pairs of question (user) and answer (bot)
    for i in range(len(conversation_history) - 1):
        question = conversation_history[i]
        answer = conversation_history[i + 1]
        conversations.append({"question": question, "answer": answer})

In [5]:
# Tokenize and format data for training
input_data = [f"User: {pair['question']} Bot: {pair['answer']}" for pair in conversations]
tokenized_data = tokenizer(input_data, return_tensors="pt", padding=True, truncation=True)

# Prepare input and target tensors for training
input_ids = tokenized_data["input_ids"][:, :-1]  # Input tensor (remove last token)
target_ids = tokenized_data["input_ids"][:, 1:]   # Target tensor (shifted by one position)

In [6]:
# Create PyTorch Dataset
class ChatDataset(Dataset):
    def __init__(self, input_ids, attention_mask, target_ids):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.target_ids = target_ids

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "target_ids": self.target_ids[idx]
        }

In [7]:
# Create DataLoader
chat_dataset = ChatDataset(input_ids, tokenized_data["attention_mask"][:, :-1], target_ids)
dataloader = DataLoader(chat_dataset, batch_size=16, shuffle=True)

# Define model and training parameters
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# Training loop
for epoch in range(3):
    for batch in dataloader:
        input_ids, attention_mask, target_ids = batch["input_ids"], batch["attention_mask"], batch["target_ids"]

        # Ensure PyTorch tensors are on the same device as the model
        input_ids, attention_mask, target_ids = input_ids.to(model.device), attention_mask.to(model.device), target_ids.to(model.device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=target_ids)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch [{epoch+1}/3] - Loss: {loss.item():.4f}")

# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/NLP PROJECT/Topical-Chat")

Epoch [1/3] - Loss: 10.0328
Epoch [1/3] - Loss: 8.9712
Epoch [1/3] - Loss: 8.1688
Epoch [1/3] - Loss: 7.2786
Epoch [1/3] - Loss: 6.4914
Epoch [1/3] - Loss: 6.0444
Epoch [1/3] - Loss: 5.4748
Epoch [1/3] - Loss: 5.0598
Epoch [1/3] - Loss: 4.6682
Epoch [1/3] - Loss: 3.9823
Epoch [1/3] - Loss: 3.9444
Epoch [1/3] - Loss: 3.3968
Epoch [1/3] - Loss: 2.7503
Epoch [1/3] - Loss: 3.0966
Epoch [1/3] - Loss: 3.2440
Epoch [1/3] - Loss: 3.0043
Epoch [1/3] - Loss: 2.8548
Epoch [1/3] - Loss: 2.4632
Epoch [1/3] - Loss: 3.0604
Epoch [1/3] - Loss: 2.5225
Epoch [1/3] - Loss: 2.2391
Epoch [1/3] - Loss: 2.3279
Epoch [1/3] - Loss: 2.8997
Epoch [1/3] - Loss: 2.3516
Epoch [1/3] - Loss: 2.6180
Epoch [1/3] - Loss: 2.7983
Epoch [1/3] - Loss: 2.4468
Epoch [1/3] - Loss: 2.5548
Epoch [1/3] - Loss: 2.2303
Epoch [1/3] - Loss: 2.3787
Epoch [1/3] - Loss: 2.4175
Epoch [1/3] - Loss: 2.4310
Epoch [1/3] - Loss: 2.5946
Epoch [1/3] - Loss: 2.5098
Epoch [1/3] - Loss: 2.6174
Epoch [1/3] - Loss: 2.3862
Epoch [1/3] - Loss: 2.7222


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("/path/to/save/fine-tuned/model")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Function to generate a response from the chatbot
def generate_response(user_input):
    input_text = f"User: {user_input} Bot:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate response using the fine-tuned GPT-2 model
    output_ids = model.generate(input_ids, max_length=50, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return response

# Example usage:
user_input = "How does your model work?"
response = generate_response(user_input)
print("Chatbot Response:", response)